In [1]:
# Import the required modules
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the FLAN-T5 large tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl").to('cuda')  # Move the model to GPU

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
# Define the input text
input_text1 = 'Classify as positive, negative, or neutral: "urgent crisis has not received the international response it merits. we can, and must, do more.\n\nthat includes passing the global food security reauthorization act of 2022. since it was passed in 2016, the bipartisan gfsa has played a key role in addressing the root causes of food insecurity and"'

input_text2 = 'Classify as positive, negative, or neutral: "groups wrote.  \u201cthis is especially relevant as organized labor is lobbying for more power through unprecedented increases of civil penalties in the h.r. 5376, the build back better act and h.r. 842, the devastating protecting the right to organize (pro) act,\u201d they"'

input_text3 = 'Classify as positive, negative, or neutral: "updates to 45l and 179d.  the inflation reduction act extends energy efficiency building incentives  in august 2022, president biden signed the inflation reduction act of 2022, h.r. 5376 . the legislation includes components of the former build back better act and addresses energy, health, and targeted tax"'

# Tokenize the input text
input_ids = tokenizer(input_text3, return_tensors="pt").input_ids.to('cuda')  # Move the input tensors to GPU

# Generate the translation output
# outputs = model.generate(input_ids, max_new_tokens=3, num_beams=3, num_return_sequences=3, return_dict_in_generate=True, output_scores=True)
outputs = model.generate(input_ids, max_new_tokens=3)

In [35]:
print(tokenizer.decode(outputs))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\bessex\AppData\Local\Temp\ipykernel_27124\3352704333.py:1 in <module>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\bessex\\AppData\\Local\\Temp\\ipykernel_27124\\3352704333.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Tensor' object has no attribute 'scores'

In [32]:
# Extract the token ids and logits
output_token_ids = outputs.sequences
output_logits = outputs.scores

# Extract the raw confidence scores for the top 3 predictions
raw_scores = [torch.softmax(output_logits[i], dim=-1)[0, output_token_ids[i, -1]].item() for i in range(output_token_ids.size(0))]

# Normalize the confidence scores so that they sum to 1
normalized_scores = [score / sum(raw_scores) for score in raw_scores]

# Iterate through each prediction and print the normalized confidence scores
for i in range(output_token_ids.size(0)):
    # Decode and print the predicted text
    predicted_text = tokenizer.decode(output_token_ids[i])
    print("Predicted text:", predicted_text)

    # Print the normalized confidence score for the prediction
    print("Normalized confidence score:", normalized_scores[i])
    print("------")

Predicted text: <pad> neutral</s>
Normalized confidence score: 6.069638130282044e-06
------
Predicted text: <pad> positive</s>
Normalized confidence score: 0.867680474658591
------
Predicted text: <pad> negative</s>
Normalized confidence score: 0.1323134557032787
------
